## Write cache GCM met (temperature and preciptation)

In [1]:
from loca import print_date
print_date()

Last executed: 2020-05-25 16:12:39.669194 by jvano on casper02


In [2]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt
# import cmocean
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature

import xarray as xr

from loca.data_catalog import load_monthly_cmip_met_datasets, resample_monthly_data

from loca.utils import calc_change

read_cache = False
# cache_dir = os.environ['TMPDIR']
# cache_dir = '/glade/u/home/jvano/scratch/'  #old
cache_dir = '/glade/scratch/jvano'  #new in 2020

# CACHED:
# models = ['ACCESS1-0', 'CanESM2']
# models = ['MIROC5']
# models = ['MRI-CGCM3']
# models = ['bcc-csm1-1']
# models = ['bcc-csm1-1-m']
# models = ['CCSM4']
# models = ['CESM1-BGC']
# models = ['CMCC-CM']
# models = ['GISS-E2-R']
# models = ['HadGEM2-CC']
# models = ['HadGEM2-ES', 'inmcm4']
# models = ['IPSL-CM5A-MR']
# models = ['MIROC-ESM', 'MIROC-ESM-CHEM']
# models = ['MPI-ESM-LR']
# models = ['MPI-ESM-MR']
# models = ['GFDL-ESM2G'] 
# models = ['NorESM1-M']
# models = ['MIROC5', 'MRI-CGCM3', 'bcc-csm1-1', 'bcc-csm1-1-m', 'CCSM4']
# models = ['CESM1-BGC', 'CESM1-CAM5', 'CMCC-CM', 'GISS-E2-R', 'HadGEM2-CC','HadGEM2-ES', 'inmcm4']
# models = ['IPSL-CM5A-MR']
# models = ['MIROC-ESM', 'MIROC-ESM-CHEM']
# models = ['MPI-ESM-LR', 'MPI-ESM-MR', 'NorESM1-M']
# models = ['GFDL-ESM2G']
# models = ['CNRM-CM5']
# models = ['CSIRO-Mk3-6-0']
# models = ['GFDL-ESM2M'] 
# models = ['GFDL-CM3'] 
# models = ['HadGEM2-AO']


# list of 25, without issues but CCSM4 and GISS-E2-R are not used in analysis:
# models = ['ACCESS1-0', 'CanESM2', 'MIROC5', 'MRI-CGCM3', 'bcc-csm1-1', 'bcc-csm1-1-m', 'CCSM4',
#           'CESM1-BGC', 'CMCC-CM', 'GISS-E2-R', 'HadGEM2-CC', 'HadGEM2-ES', 'inmcm4', 'IPSL-CM5A-MR',
#           'MIROC-ESM', 'MIROC-ESM-CHEM', 'MPI-ESM-LR', 'MPI-ESM-MR', 'NorESM1-M', 'GFDL-ESM2G', 
#           'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-ESM2M','GFDL-CM3', 'HadGEM2-AO']

# list of 23:
models = ['ACCESS1-0', 'CanESM2', 'MIROC5', 'MRI-CGCM3', 'bcc-csm1-1', 'bcc-csm1-1-m',
          'CESM1-BGC', 'CMCC-CM', 'HadGEM2-CC', 'HadGEM2-ES', 'inmcm4', 'IPSL-CM5A-MR',
          'MIROC-ESM', 'MIROC-ESM-CHEM', 'MPI-ESM-LR', 'MPI-ESM-MR', 'NorESM1-M', 'GFDL-ESM2G', 
          'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-ESM2M','GFDL-CM3', 'HadGEM2-AO']

# models = ['ACCESS1-0']
# models = ['MIROC5', 'MRI-CGCM3', 'bcc-csm1-1', 'bcc-csm1-1-m', 'CCSM4']
# models = ['CESM1-BGC', 'CMCC-CM', 'GISS-E2-R', 'HadGEM2-CC', 'inmcm4', 'IPSL-CM5A-MR']
# models = ['MIROC-ESM', 'MIROC-ESM-CHEM', 'MPI-ESM-LR', 'MPI-ESM-MR', 'NorESM1-M', 'GFDL-ESM2G'] 
# models = ['CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-ESM2M','GFDL-CM3', 'HadGEM2-AO']
models = ['CanESM2','HadGEM2-ES']

In [3]:
from dask.distributed import Client
client = Client(scheduler_file=os.path.join(os.environ['HOME'], 'scheduler_file.json'))
client

Client Scheduler: tcp://10.12.205.11:38333 Dashboard: proxy/46152/status,Cluster Workers: 2 Cores: 4 Memory: 60.00 GB


In [48]:
#Historical

# hist_cmip_met_data = {}
# if read_cache:
#     print("reading values from cache")
#     for key in ['loca', 'bcsd']:
#         hist_cmip_met_data[key] = xr.open_mfdataset(
#             os.path.join(os.environ['TMPDIR'], f'monthly_cmip_met_hist.{key}_*.nc'),
#             concat_dim='gcm', chunks={'time': 72})
#     print("finished reading from cache")
#     ds = hist_cmip_met_data['loca']
#     print(ds['pcp'])
#     #TODO once get obs load separate won't have to do work around below
#     single = ['ACCESS1-0']
#     obs_data = load_monthly_historical_met_datasets(models=single, autoclose=True, parallel=True)
#     print("finished reading obs")

# else:
#     print("reading values from files directly")
#     met_data = load_monthly_historical_met_datasets(models=models, autoclose=True, parallel=True)

In [4]:
write_cache = True  
models

if write_cache:
    xr.set_options(file_cache_maxsize=12000)
    hist_data = load_monthly_cmip_met_datasets('historical', models=models, autoclose=False, parallel=True)
    rcp4_data = load_monthly_cmip_met_datasets('rcp45', models=models, autoclose=False, parallel=True)
    rcp8_data = load_monthly_cmip_met_datasets('rcp85', models=models, autoclose=False, parallel=True)

load_monthly_cmip_met_datasets
load_monthly_loca_meteorology
load_daily_loca_meteorology
load_monthly_bcsd_meteorology
load_bcsd_dataset
load_monthly_cmip_met_datasets
load_monthly_loca_meteorology
load_daily_loca_meteorology
load_monthly_bcsd_meteorology
load_bcsd_dataset
load_monthly_cmip_met_datasets
load_monthly_loca_meteorology
load_daily_loca_meteorology
load_monthly_bcsd_meteorology
load_bcsd_dataset


In [5]:
if write_cache:
    for key, ds in hist_data.items():
        for gcm in models:
            ds.sel(gcm=gcm).load().to_netcdf(os.path.join(os.environ['TMPDIR'], f'monthly_cmip_met_hist.{key}_{gcm}.nc'))
    for key, ds in rcp4_data.items():
        for gcm in models:
            ds.sel(gcm=gcm).load().to_netcdf(os.path.join(os.environ['TMPDIR'], f'monthly_cmip_met_rcp4.{key}_{gcm}.nc'))
    for key, ds in rcp8_data.items():
        for gcm in models:
            ds.sel(gcm=gcm).load().to_netcdf(os.path.join(os.environ['TMPDIR'], f'monthly_cmip_met_rcp8.{key}_{gcm}.nc'))
    for key, ds in obs_data.items():
            ds.sel()

distributed.client - WARNING - Couldn't gather 386 keys, rescheduling {"('getitem-db73da2a746fce6b2ef911884e7f1980', 383, 0, 0)": ('tcp://10.12.205.30:33667',), "('getitem-db73da2a746fce6b2ef911884e7f1980', 553, 0, 0)": ('tcp://10.12.205.27:35753',), "('getitem-db73da2a746fce6b2ef911884e7f1980', 375, 0, 0)": ('tcp://10.12.205.30:33667',), "('getitem-d21c3dafd1a36e44e6ac84020ba5de22', 174, 0, 0)": ('tcp://10.12.205.30:33667',), "('getitem-a8307eac4dbeea4a1c61628d52865728', 557, 0, 0)": ('tcp://10.12.205.28:39735',), "('getitem-db73da2a746fce6b2ef911884e7f1980', 537, 0, 0)": ('tcp://10.12.205.28:39735',), "('getitem-d21c3dafd1a36e44e6ac84020ba5de22', 178, 0, 0)": ('tcp://10.12.205.30:33667',), "('getitem-f02dcde28ecea872ab413baddbf94f94', 31, 0, 0)": ('tcp://10.12.205.28:36301',), "('getitem-f02dcde28ecea872ab413baddbf94f94', 560, 0, 0)": ('tcp://10.12.205.27:35753',), "('getitem-a8307eac4dbeea4a1c61628d52865728', 516, 0, 0)": ('tcp://10.12.205.27:35753',), "('getitem-f02dcde28ecea872ab4

NameError: name 'obs_data' is not defined

In [7]:
#test to see if reads in

read_cache = True
hist_data = {}
rcp4_data = {}
rcp8_data = {}
if read_cache:
    for key in ['loca', 'bcsd']:
        hist_data[key] = xr.open_mfdataset(
            os.path.join(cache_dir, f'monthly_cmip_met_hist.{key}_*.nc'),
            concat_dim='gcm', chunks={'time': 72})
        rcp4_data[key] = xr.open_mfdataset(
            os.path.join(cache_dir, f'monthly_cmip_met_rcp4.{key}_*.nc'),
            concat_dim='gcm', chunks={'time': 72})
        rcp8_data[key] = xr.open_mfdataset(
            os.path.join(cache_dir, f'monthly_cmip_met_rcp8.{key}_*.nc'),
            concat_dim='gcm', chunks={'time': 72})

In [10]:
# rcp4_data
# rcp8_data
hist_data

{'loca': <xarray.Dataset>
 Dimensions:  (gcm: 23, lat: 224, lon: 464, time: 672)
 Coordinates:
   * time     (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2005-12-01
   * lon      (lon) float64 -124.9 -124.8 -124.7 -124.6 ... -67.31 -67.19 -67.06
   * lat      (lat) float64 25.06 25.19 25.31 25.44 ... 52.56 52.69 52.81 52.94
   * gcm      (gcm) <U14 'ACCESS1-0' 'CESM1-BGC' ... 'bcc-csm1-1' 'inmcm4'
 Data variables:
     pcp      (gcm, time, lat, lon) float64 dask.array<shape=(23, 672, 224, 464), chunksize=(1, 72, 224, 464)>
     t_max    (gcm, time, lat, lon) float32 dask.array<shape=(23, 672, 224, 464), chunksize=(1, 72, 224, 464)>
     t_min    (gcm, time, lat, lon) float32 dask.array<shape=(23, 672, 224, 464), chunksize=(1, 72, 224, 464)>
     t_mean   (gcm, time, lat, lon) float32 dask.array<shape=(23, 672, 224, 464), chunksize=(1, 72, 224, 464)>,
 'bcsd': <xarray.Dataset>
 Dimensions:  (gcm: 23, lat: 222, lon: 462, time: 672)
 Coordinates:
   * time     (time) datetime64[ns] 195